In [6]:
using CSV, DataFrames, Gadfly, Distributions, SparseArrays, LinearAlgebra, Mamba, ForwardDiff

include("iGMRF.jl");
include("dataGen.jl");

# Création de la grille à 9 cellules

Chaque cellule est représentée par un triplet $[\mu, \sigma, \xi]$ indiquant les vraie valeur des paramètres de la GEV à déterminer.

In [7]:
# Seed
Random.seed!(400)
# Dimensions de la grille
m₁ = 3;
m₂ = 3;
# Nb total de cellules
m = m₁ * m₂
# Insuffisance de rang de la matrice de structure
r = 1;
# Hyperparamètres
κᵤ = 100.0;
# Matrice de structure
F = iGMRF(m₁, m₂, κᵤ);

In [47]:
grid_target = generateTargetGrid(F) .+ 10;

In [48]:
grid_target[:, :, 1]

3×3 Matrix{Float64}:
 9.88554  10.0053   10.1517
 9.92388   9.96065  10.1539
 9.85489   9.99319  10.071

# Génération de données

Le but est de générer des données sur chaque cellule à partir des paramètres de la GEV de la grille cible.

In [49]:
nobs = 1000
data = generateData(grid_target, nobs);

# Modèle 

In [50]:
## Log-transformed Posterior(μ | κᵤ=100) + Constant and Gradient Vector
logπ̃grad = function(μ::DenseVector)
  logπ̃(μ::DenseVector) = sum(loglikelihood.(GeneralizedExtremeValue.(μ, 1, 0), data)) - 100 * μ' * F.G.W * μ / 2
  grad = ForwardDiff.gradient(logπ̃, μ)
  logπ̃(μ), grad
end

n = 10000
sim1 = Chains(n, m, names = ["μ1", "μ2", "μ3", "μ4", "μ5", "μ6", "μ7", "μ8", "μ9"])
sim2 = Chains(n, m, names = ["μ1", "μ2", "μ3", "μ4", "μ5", "μ6", "μ7", "μ8", "μ9"])
epsilon = 0.001
Sigma = Matrix{Float64}(I, m, m)
theta1 = MALAVariate(zeros(m), epsilon, logπ̃grad)
theta2 = MALAVariate(zeros(m), epsilon, Sigma, logπ̃grad)
for i in 1:n
  sample!(theta1)
  sample!(theta2)
  sim1[i, :, 1] = theta1
  sim2[i, :, 1] = theta2
end

μ̂₁ = mean(sim1.value, dims=1);
μ̂₂ = mean(sim2.value, dims=1);

In [51]:
draw(plot(sim1), fmt=:svg, filename="plots/sim1.svg", nrow=9, height=24inch)

In [52]:
reshape(μ̂₁, m₁, m₂)'

3×3 adjoint(::Matrix{Float64}) with eltype Float64:
 9.82989  9.98427  10.0526
 9.82943  9.91811  10.1377
 9.83266  9.93553  10.0426

In [53]:
grid_target[:, :, 1]

3×3 Matrix{Float64}:
 9.88554  10.0053   10.1517
 9.92388   9.96065  10.1539
 9.85489   9.99319  10.071

Distance entre les deux matrices

In [54]:
norm(reshape(μ̂₁, m₁, m₂)' .- grid_target[:, :, 1], 2) / m

0.018915223113039645